# Conditional Expectation

<!-- SUMMARY: Normal transform (Gaussian anamorphosis). Deriving conditional expectation of exceeding a threshold  -->

<!-- CATEGORY: Courses -->

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc
import matplotlib.pyplot as plt
import numpy as np
import os

gdoc.setNoScroll()

figsize = (10,8)

In [ ]:
ndim = 2
gl.defineDefaultSpace(gl.ESpaceType.RN, ndim);

In [ ]:
## Load observations
temp_nf = gdoc.loadData("Scotland", "Scotland_Temperatures.NF")
dat = gl.Db.createFromNF(temp_nf)

## Load grid
elev_nf = gdoc.loadData("Scotland", "Scotland_Elevations.NF")
grid = gl.DbGrid.createFromNF(elev_nf)

In [ ]:
unique_neigh = gl.NeighUnique.create()

Histogram of the raw variable (Temperature)

In [ ]:
gp.histogram(dat, name="January*", bins=20)
gp.decoration(title="Temperatures")

Gaussian scores

In [ ]:
anam = gl.AnamHermite(30)
err = anam.fitFromLocator(dat)
err = anam.rawToGaussian(dat, "January_temp")

In [ ]:
anam.display()

Plot the Gaussian scores

In [ ]:
gp.sortedCurve(tabx=dat["Y.January_temp"], taby=dat["January_temp"])
gp.decoration(xlabel="Gaussian",ylabel="Raw")

Draw the histogram of the Gaussian transformed values

In [ ]:
gp.histogram(dat, name="Y.January*", bins=20)
gp.decoration(title="Temperatures (Gaussian scale)")

We calculate the experimental directional variogram of the gaussian scores and fit the Model (with the constraints that sill should be 1)

In [ ]:
varioparam = gl.VarioParam.createMultiple(ndir=2, nlag=40, dlag=10)
vario_gauss2dir = gl.Vario.create(varioparam)
err = vario_gauss2dir.compute(dat)

fitmodgauss = gl.Model()
err = fitmodgauss.fit(vario_gauss2dir, 
                types=[gl.ECov.NUGGET, gl.ECov.SPHERICAL, gl.ECov.CUBIC],
                constraints = gl.Constraints(1))

In [ ]:
res = gp.varmod(vario_gauss2dir, fitmodgauss)

In [ ]:
neighU = gl.NeighUnique.create()

Kriging of Gaussian scores

In [ ]:
err = gl.kriging(dat, grid, fitmodgauss, neighU)

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
ax.geometry(aspect=1)
ax.raster(grid, "*estim")
ax.symbol(dat)
ax.decoration(title="Kriging of Gaussian scores")

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
ax.geometry(aspect=1)
ax.raster(grid, "*stdev")
ax.symbol(dat, flagCst=True)
ax.decoration(title="St. Dev. of Gaussian scores")

## Conditional Expectation

We use the Monte Carlo method with 1000 outcomes.

In [ ]:
selectivity = gl.Selectivity.createByKeys(["Z"], [], flag_est=True, flag_std=True)
err = gl.ConditionalExpectation(grid, anam, selectivity, "K*.estim", "K*.stdev", nbsimu=100,
                                namconv=gl.NamingConvention("CE",False,True,False))

Display of the Conditional Expectation

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
ax.geometry(aspect=1)
ax.raster(grid, "CE*estim")
ax.symbol(dat)
ax.decoration(title = "Conditional Expectation")

Display of the Conditional Standard Deviation

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
ax.geometry(aspect=1)
ax.raster(grid, "CE*stdev")
ax.symbol(dat, flagCst=True)
ax.decoration(title="Conditional Standard Deviation")

Conditional Probability below 0

In [ ]:
selectivity = gl.Selectivity.createByKeys(["PROP"], zcuts=[0],flag_est=True, flag_std=True)
err = gl.ConditionalExpectation(grid, anam, selectivity, "K*.estim", "K*.stdev",
                                namconv=gl.NamingConvention("CE",False,True,False))

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
ax.geometry(aspect=1)
ax.raster(grid, "CE.Proba*estim")
ax.symbol(dat)
ax.decoration(title = "Conditional Probability below 0")

Conditional Probability above 1

In [ ]:
selectivity = gl.Selectivity.createByKeys(["T"], zcuts=[1],flag_est=True, flag_std=True)
err = gl.ConditionalExpectation(grid, anam, selectivity, "K*.estim", "K*.stdev",
                                namconv=gl.NamingConvention("CE",False,True,False))

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
ax.geometry(aspect=1)
ax.raster(grid, "CE.T*estim-1")
ax.symbol(dat)
ax.decoration(title = "Conditional Probability above 1")

In [ ]:
fig, ax = plt.subplots(figsize=figsize)
ax.geometry(aspect=1)
ax.raster(grid, "CE.T*stdev-1")
ax.symbol(dat, flagCst=True)
ax.decoration(title = "Conditional probability (Standard Deviation)")